# Gerekli Kütüphanelerin Eklenmesi 

In [2]:
#Gerekli kütüphaneleri projeye ekleyerek başlıyoruz. 
import pandas as pd #veri işlemleri için pandas kütüphanesini kullanırız
import numpy as np  #sayısal işlemler için numpy kullanacağız
import matplotlib.pyplot as plt #grafik işlemleri için 
from sklearn.model_selection import train_test_split #veri bölme
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler #veri ölçeklendirme

from sklearn.svm import SVR #SVR modeli ---> benim ödevdeki algoritmam 
from sklearn.linear_model import LinearRegression #lineer regresyon 
from sklearn.preprocessing import PolynomialFeatures #Polinom regresyon 
from sklearn.metrics import mean_absolute_error, r2_score #Değerlendirme metrikleri

#grafiklerin notebook içinde görünmesini sağlamak için aşağıdaki komutu yazıyoruz
%matplotlib inline 




## Veri Önişleme 

_Bir ihtimal veri url yolu ile okunması gerekirse_

In [3]:
#url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv"
#df = pd.read_csv(url)

* Veri Setini Okuma Ve İlk 5 Veriyi Yazdırma / Eksik Değerleri Kontrol

In [ ]:
#Verileri okuyacağız 
file_path = "data/forest_fires.csv" #veri setinin yolunu yazıyoruz. 
data = pd.read_csv(file_path, skiprows=1, header=0, encoding='utf-8') #Burada dosyayı okuyoruz. 
print("Veri Setinin Ilk 5 Satırı:") #veri setinde bulunan ilk 5 veriyi çekiyoruz
display(data.head()) #ilk 5 veriyi çekecek komut

print("\nVeri Setinin Bilgisi:")
data.info() #veri seti hakkında genel bilgiyi bize yazacak

print("\nEksik Değerler:")
print(data.isnull().sum()) #eksik değerleri kontol edeceğiz ve print ile yazdıracağız


* Eksik Değerlerin Temizlenmesi/Düzeltilmesi 

In [ ]:
#ilk denemem de veri türlerini dikkate almadığım için hata aldım
#veri türlerini kontrol ederek sütunları inceleyelim 
print("\n Sütunlar ve veri türleri")
print(data.dtypes) #bu komut ile veri türlerini kontrol ediyoruz :)

#şimdi eksik değerleri ortalama ile dolduracağımız için sadece sayısal değerleri alıyoruz
numeric_columns = data.select_dtypes(include=[np.number]).columns 

#eksik sütunları ortalama alarak tamamlıyoruz
print("\n Eksik değerlerin doldurulması")
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())

#Eksik değerler doldurulduktan sonrası
print("\n Sonrası")
print(data.isnull().sum())
print("\n İlk 5 veri")
display(data.head())

* Öznitelik Seçimi 

In [ ]:
#Ödev de bana "ilk 1-6 arası öznitelikler kullanılarak modelleme yapılacaktır " şeklinde bir ibare verildiği için buna dikkate alarak öznitelik seçimlerini yapacağım
# ilk 1-6 sütun bağımsız değişkenler yani x olacak 
x = data.iloc[:, 1:7]

#Bağımlı değişken olarak FWI sütununu seçeceğim
y = data['FWI']

#Seçimleri doğrulamak için verilerin boyutunu kontrol edeceğiz
print("\n Bağımsız Değişken Boyutu", x.shape)
print("\n Bağımlı Değişken Boyutu", y.shape )

#ilk 5 satır görüntüleme 
print("\n Bağımsız Değişkenler:")
display(x.head())
print("\n Bağımlı Değişken:")
display(y.head())



## Verinin Eğitim ve Test Setlerine Ayrılması 

In [ ]:
#Veri setini eğitim ve test verisi olarak ikiye böleceğiz. 
#Ben %70 eğitim - %30 test verisi olarak ayıracağım fakat %80 eğitim - %20 test şeklinde de bölünebilir
#Ben projenin performansını ölçmek için 70-30 seçtim. Model daha iyi öğrensin istersek 80-20 seçilebilir.

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42) #bu komut ile veri setini böldük
print("\nEğitim Seti Boyutu:")
print("x_train:", x_train.shape, "y_train:", y_train.shape) #shape ile görüntüleme işlemi yapacağız
print("\nTest Seti Boyutu:")
print("x_test:", x_test.shape, "y_test:", y_test.shape)

* Veri Ölçeklendirme

In [43]:
#StandardScaler, veri setindeki özelliklerin(features) standartlaştırılmasını sağlar. 
#sklearn.preprocessing modülünde yer alır
#veriyi hazırlamak için kullanacağız 
#Bunu kullanmak bize 2 katkı sağlayacak
#1. Modelin daha hızlı ve daha doğru öğrenmesi sağlanır.
#2. Özniteliklerin (features) farklı ölçekleri arasında dengesizlik önlenir.

#SVR modelinde veri ölçeklendirme yapacağız. Bunu yaparken StandardScaler kullanarak normalleştirme yapacağız.
#ilk yazdığım kodda sayısal olmayan değişkenleri hesaba katmamıştım. 
#StandardScaler yalnızca sayısal değerler üzerinde çalışır.

#Sayısal sütunlara dönüştürme işlemi yapalım sonrasında NaN değerler için kontrol ve doldurma işlemi yapalım
x_train = x_train.apply(pd.to_numeric, errors='coerce')
x_test = x_test.apply(pd.to_numeric, errors='coerce')
x_train = x_train.fillna(x_train.mean())#fillna() fonksiyonu ile eksik değerler doldurulur
x_test = x_test.fillna(x_test.mean())

scaler = StandardScaler() #Ölçeklendirme işlemine burada başlıyoruz

x_train_scaled = scaler.fit_transform(x_train) #eğitim seti ölçeklendirme
x_test_scaled = scaler.transform(x_test) #test setini ölçeklendirme işlemi

print("\n Ölçeklendirme Eğitim Seti Boyutu", x_train_scaled.shape)
print("\n Ölçeklendirilmiş Test Seti Boyutu:", x_test_scaled.shape)


 Ölçeklendirme Eğitim Seti Boyutu (172, 6)

 Ölçeklendirilmiş Test Seti Boyutu: (74, 6)
